In [ ]:
import json
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import numpy as np
import os
import sys
import matplotlib.image as mpimg
import glob
import time
import math
from collections import defaultdict
import datetime
import warnings
import glob
from numpy.random import default_rng
from PIL import Image
import copy
import pickle as pkl

%matplotlib inline

DATA_PATH = '' ##Add path to images here
warnings.filterwarnings("ignore")

In [ ]:
def save_pkl(obj, filename):
    with open(filename, 'wb') as f:
        pkl.dump(obj, f)
#     return obj4

In [ ]:
def get_coordinates_of_all_points(pd_series):
    x_coords = []
    y_coords = []
    for col_name in pd_series.index:
        if col_name[-2:]=='_x':
            x_coords.append(pd_series[col_name])
        elif col_name[-2:]=='_y':
            y_coords.append(pd_series[col_name])
    return x_coords, y_coords
            

In [ ]:
def create_poly_line(points, degree=5):
    x,y=zip(*points)
    lspace=np.linspace(min(x),max(x),100)
    z=np.polyfit(x,y,degree) ## Fit a 5-dimensional line through the midpoints to mark the axis
    
    return z, lspace

In [ ]:
def create_poly_line_head(points):
    x,y=zip(*points)
    lspace=np.linspace(min(y),max(y),100)
    z=np.polyfit(y,x,3) ## Fit a 5-dimensional line through the midpoints to mark the axis
    
    return z, lspace

In [ ]:
def get_relevant_x_y_coords(series, col_names):
    points = []
    for col_name in col_names:
        x = series[col_name[0]]
        y = series[col_name[1]]
        points.append([x,y])
    return points

In [ ]:
def draw_poly_line(im, z, lspace):
    draw_x=lspace
    draw_y=np.polyval(z,draw_x)
    draw_points = (np.asarray([draw_x, draw_y]).T).astype(np.int32)
    im=cv2.polylines(im, [draw_points], False, (255,255,255),thickness=2) ## Draw the axis
    
    return im, draw_points

In [ ]:
def draw_poly_line_head(im, z, lspace):
    draw_y=lspace
    draw_x=np.polyval(z,draw_y)
    draw_points = (np.asarray([draw_x, draw_y]).T).astype(np.int32)
    im=cv2.polylines(im, [draw_points], False, (255,255,255),thickness=2) ## Draw the axis
    
    return im, draw_points

In [ ]:
def get_slope(points):
    x1 = points[0][0]
    x2 = points[1][0]
    y1 = points[0][1]
    y2 = points[1][1]
    m = (y2-y1)/(x2-x1)
    
    return m

In [ ]:
def get_slope_with_flipped_coords(points):
    x1 = points[0][0]
    x2 = points[1][0]
    y1 = points[0][1]
    y2 = points[1][1]
    m = (x2-x1)/(y2-y1)
    
    return m

In [ ]:
def increase_width(points, percentage=10):
    x1 = points[0][0]
    x2 = points[1][0]
    y1 = points[0][1]
    y2 = points[1][1]
#     m = (y2-y1)/(x2-x1)
    m = get_slope(points) 
    c = y1-m*x1
    
    dist = np.sqrt((y2-y1)**2 + (x2-x1)**2)
    
    delta = dist*percentage/100 ## Increase by 5 percent
    
    if y1>y2:
        y1_new = y1 + delta
        y2_new = y2 - delta
        x1_new = (y1_new-c)/m
        x2_new = (y2_new-c)/m
    else:
        y1_new = y1 - delta
        y2_new = y2 + delta
        x1_new = (y1_new-c)/m
        x2_new = (y2_new-c)/m
        
    return [(x1_new, y1_new), (x2_new, y2_new)]

In [ ]:
def get_segmentation_mask(z, first_bound, second_bound):
    points = [first_bound, second_bound]
    x,y=zip(*points)
    if x[0]>x[1]:
        lspace=np.linspace(x[1],x[0],100)
    else:
        lspace=np.linspace(x[0],x[1],100)
    draw_x = lspace
    draw_y=np.polyval(z,draw_x)
    draw_points = (np.asarray([draw_x, draw_y])).astype(np.int32)
    
    return draw_points

In [ ]:
def check_vertical_whales(series):
    required_points = [('rostrum_x', 'rostrum_y'),
                      ('fluke_x', 'fluke_y')]
    x_y_coords = get_relevant_x_y_coords(series, required_points)
    slope = get_slope(x_y_coords)
#     print(slope)
    if slope>1 or slope<-1:
        return 1
    return 0

In [ ]:
def transform_points_for_vertical_whales(series):
    new_series = copy.deepcopy(series)
    for col_name in series.index:
        if col_name[-2:]=='_x':
            new_series[col_name] = series['Image.Length']-series[col_name[:-2]+'_y']
        elif col_name[-2:]=='_y':
            new_series[col_name] = series[col_name[:-2]+'_x']
    return new_series

In [ ]:
def transform_final_seg_mask(seg_mask, image_length):
    new_x = []
    new_y = []
    
    for x in seg_mask[0]:
        new_y.append(image_length - x)
    for y in seg_mask[1]:
        new_x.append(y)
    
    transformed_mask = [new_x,new_y]
    
    return np.asarray(transformed_mask)

In [ ]:
# Draw 3 poly lines, one each for:

# # 'side1_head1_x', 'side1_head1_y', 
# # 'side1_head2_x', 'side1_head2_y', 
# # 'side1_eye_x', 'side1_eye_y', 
# # 'side1_body1_x', 'side1_body1_y', 
# # 'side1_body2_x', 'side1_body2_y', 
# # 'side1_body3_x', 'side1_body3_y', 
# # 'side1_peduncle_width_x', 'side1_peduncle_width_y',

# # 'side2_head1_x', 'side2_head1_y',
# # 'side2_head2_x', 'side2_head2_y',
# # 'side2_eye_x', 'side2_eye_y',
# # 'side2_body1_x', 'side2_body1_y',
# # 'side2_body2_x', 'side2_body2_y', 
# # 'side2_body3_x', 'side2_body3_y',
# # 'side2_peduncle_width_x','side2_peduncle_width_y', 


# # 'side1_head1_x', 'side1_head1_y', 
# # 'rostrum_x', 'rostrum_y',
# # 'side2_head1_x', 'side2_head1_y',


def get_final_segmentation_masks(im, series):
    side1_col_names = [('side1_head1_x', 'side1_head1_y'), 
                        ('side1_head2_x', 'side1_head2_y'), 
                        ('side1_eye_x', 'side1_eye_y'), 
                        ('side1_body1_x', 'side1_body1_y'), 
                        ('side1_body2_x', 'side1_body2_y'), 
                        ('side1_body3_x', 'side1_body3_y'), 
                        ('side1_peduncle_width_x', 'side1_peduncle_width_y')]
    side2_col_names = [('side2_head1_x', 'side2_head1_y'), 
                        ('side2_head2_x', 'side2_head2_y'), 
                        ('side2_eye_x', 'side2_eye_y'), 
                        ('side2_body1_x', 'side2_body1_y'), 
                        ('side2_body2_x', 'side2_body2_y'), 
                        ('side2_body3_x', 'side2_body3_y'), 
                        ('side2_peduncle_width_x', 'side2_peduncle_width_y')]
    head_col_names = [#('side1_head2_x', 'side1_head2_y'),
                    ('side1_head1_x', 'side1_head1_y'), 
                    ('rostrum_x', 'rostrum_y'),
                    ('side2_head1_x', 'side2_head1_y'),]
                     #('side2_head2_x', 'side2_head2_y'),]
    
    eye_col_names = [('side1_eye_x', 'side1_eye_y'), 
                 ('side2_eye_x', 'side2_eye_y')]
    
    head1_col_names = [('side1_head1_x', 'side1_head1_y'),
                      ('side2_head1_x', 'side2_head1_y'),]
    
    body2_col_names = [('side1_body2_x', 'side1_body2_y'), 
                        ('side2_body2_x', 'side2_body2_y')]
    
    peduncle_col_names = [('peduncle_x', 'peduncle_y')]
    
    fluke_col_names = [('fluke_x', 'fluke_y')]
    
    side1_peduncle_width_col_names = [('side1_peduncle_width_x', 'side1_peduncle_width_y')]
    side2_peduncle_width_col_names = [('side2_peduncle_width_x', 'side2_peduncle_width_y')]
    
    vertical_flag = 0
    
    if check_vertical_whales(series):
            vertical_flag = 1
            original_series_for_vertical_whales = copy.deepcopy(series)
            series = transform_points_for_vertical_whales(original_series_for_vertical_whales)
            im_original = copy.deepcopy(im)
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)

    points_eye = get_relevant_x_y_coords(series, eye_col_names)
    points_side1 = get_relevant_x_y_coords(series, side1_col_names)
    points_side2 = get_relevant_x_y_coords(series, side2_col_names)
    points_head = get_relevant_x_y_coords(series, head_col_names) ## head1 + rostrum
    points_head1 = get_relevant_x_y_coords(series, head1_col_names)
    points_body2 = get_relevant_x_y_coords(series, body2_col_names)
    points_peduncle =  get_relevant_x_y_coords(series, peduncle_col_names)
    points_side1_peduncle_width = get_relevant_x_y_coords(series, side1_peduncle_width_col_names)
    points_side2_peduncle_width = get_relevant_x_y_coords(series, side2_peduncle_width_col_names)
    points_fluke = get_relevant_x_y_coords(series, fluke_col_names)
    
    eye_points_increased_width = increase_width(points_eye)
    body2_points_increased_width = increase_width(points_body2)

    series.update(pd.Series([eye_points_increased_width[0][0], eye_points_increased_width[0][1],
                            eye_points_increased_width[1][0], eye_points_increased_width[1][1]], 
                            index=['side1_eye_x', 'side1_eye_y', 'side2_eye_x', 'side2_eye_y']))
    series.update(pd.Series([body2_points_increased_width[0][0], body2_points_increased_width[0][1],
                            body2_points_increased_width[1][0], body2_points_increased_width[1][1]], 
                            index=['side1_body2_x', 'side1_body2_y', 'side2_body2_x', 'side2_body2_y']))

    points_side1_new = get_relevant_x_y_coords(series, side1_col_names)
    points_side2_new = get_relevant_x_y_coords(series, side2_col_names)
    poly_func_side1, lspace_side1 = create_poly_line(points_side1_new)
    poly_func_side2, lspace_side2 = create_poly_line(points_side2_new)
    img_with_side1_line_drawn, draw_points_side1 = draw_poly_line(im, poly_func_side1, lspace_side1)
    img_with_side1_side2_lines_drawn, draw_points_side2 = draw_poly_line(img_with_side1_line_drawn, poly_func_side2, lspace_side2)

    seg_mask_side1 = get_segmentation_mask(poly_func_side1, points_head1[0], points_side1_peduncle_width[0])
    seg_mask_side2 = np.flip(get_segmentation_mask(poly_func_side2, points_head1[1], points_side2_peduncle_width[0]), axis=1)

    slope_at_side1_head1_side2_head1 = get_slope(points_head1)
    if slope_at_side1_head1_side2_head1>= -0.5 and slope_at_side1_head1_side2_head1<= 0.5:
        poly_func_head, lspace_head = create_poly_line(points_head)
        img_with_side1_side2_head_lines_drawn, draw_points_head = draw_poly_line(img_with_side1_side2_lines_drawn, poly_func_head, lspace_head)
    else:
        poly_func_head, lspace_head = create_poly_line_head(points_head)
        img_with_side1_side2_head_lines_drawn, draw_points_head = draw_poly_line_head(img_with_side1_side2_lines_drawn, poly_func_head, lspace_head)

    seg_mask_head = draw_points_head.T

    dist_head_start_seg1_end = (seg_mask_head[0][0]-seg_mask_side1[0][-1])**2 + (seg_mask_head[1][0]-seg_mask_side1[1][-1])**2
    dist_head_start_seg2_end = (seg_mask_head[0][0]-seg_mask_side2[0][-1])**2 + (seg_mask_head[1][0]-seg_mask_side2[1][-1])**2

    if dist_head_start_seg1_end<dist_head_start_seg2_end:
        final_seg_mask = np.concatenate([seg_mask_side1, seg_mask_head, seg_mask_side2], axis=1)
    else:
        final_seg_mask = np.concatenate([seg_mask_side2, seg_mask_head, seg_mask_side1], axis=1)

    points_peduncle_np = np.asarray(points_peduncle).reshape(2,1)
    final_seg_mask = np.concatenate([final_seg_mask, points_peduncle_np], axis=1)
    

    if vertical_flag:
        final_seg_mask = transform_final_seg_mask(final_seg_mask, series['Image.Length'])
        im = im_original
#     plt.figure(figsize=(10,10))
#     plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
#     plt.scatter(final_seg_mask[0], final_seg_mask[1], marker='o', color='yellow', s=10)
#         x, y = get_coordinates_of_all_points(series)
#         plt.figure(figsize=(10,10))
#         plt.imshow(cv2.cvtColor(img_with_side1_side2_head_lines_drawn, cv2.COLOR_BGR2RGB))
#         plt.scatter(x, y, marker='o', color='magenta', s=5)

    return final_seg_mask
    
#     draw_x=lspace
#     draw_y=np.polyval(z,draw_x)
#     draw_points = (np.asarray([draw_x, draw_y]).T).astype(np.int32)
#     im=cv2.polylines(im, [draw_points], False, (255,255,255),thickness=2) ## Draw the axis

In [ ]:
df = pd.read_csv('all_rows_filled_with_img_width_and_length_with_white_calves_jul_2023.csv', index_col=None)

In [ ]:
# rng = default_rng()
# random_idx = rng.choice(df.shape[0], size=20, replace=False)
# for i in random_idx:
#     img_filename = df['filename'].iloc[i]
#     x, y = get_coordinates_of_all_points(df.iloc[i])
#     image = Image.open(os.path.join(DATA_PATH,img_filename))
#     plt.figure(figsize=(10,10))
#     plt.imshow(image)
#     plt.scatter(x, y, marker='o', color='red', s=5)

In [ ]:
# rng = default_rng()
# random_idx = rng.choice(df.shape[0], size=20, replace=False)
# for i in random_idx:
#     img_filename = df['filename'].iloc[i]
#     x, y = get_coordinates_of_all_points(df.iloc[i])
#     x[-4] = x[0]
#     y[-4] = y[0]
    
#     image = Image.open(os.path.join(DATA_PATH,img_filename))
#     plt.figure(figsize=(10,10))
#     plt.imshow(image)
#     plt.plot(x[:-3], y[:-3], marker='.', color='red')

In [ ]:
all_final_seg_masks = []
count = 0
white_calves_idx = [i for i in range(597, 639)]
# for i in range(df.shape[0]):
for i in random_idx:
    count += 1
    img_filename = df['filename'].iloc[i]
    image = cv2.imread(os.path.join(DATA_PATH,img_filename))
    final_seg_mask = get_final_segmentation_masks(image, df.iloc[i])
    image_ = Image.open(os.path.join(DATA_PATH,img_filename))
    
    x, y = get_coordinates_of_all_points(df.iloc[i])
    x[-4] = x[0]
    y[-4] = y[0]
    plt.figure(figsize=(10,10))
    plt.imshow(image_)
    plt.plot(x[:-3], y[:-3], marker='.', color='yellow')
    
    plt.figure(figsize=(10,10))
    plt.imshow(image_)
    plt.plot(final_seg_mask[0], final_seg_mask[1], marker='.', color='red')
    all_final_seg_masks.append(final_seg_mask)

In [ ]:
save_pkl(all_final_seg_masks, "final_segmentation_masks.pkl")